In [1]:
from functools import reduce

In [2]:
import pandas as pd
import numpy as np
#import openpyxl
import requests
import json
import pandas as pd
from os import path
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats

The file contains the DAO v0.3 evaluation, including the HOPR forum plugins, and the votes on snapshot 
CSV file's numbers always refer to the forum plugins, from where they were extracted

The abbreviatins follow the logic of the allocation board on the drive: https://docs.google.com/spreadsheets/d/1netmY3CkYo06PxdDMv4jqifty24Y6gBnHQko0ZyVmzE/edit#gid=0 

Winning proposal 	P_W_P
Winning siniture 	P_W_S
Vote stage not winning**	P_V_P
Vote stage signitures 	P_V_S
Referendum stage not voting	P_R_P
Referendum stage not voting signitures	P_R_S
Ten posts with most support	D_P_L
Ten users with most total likes across discussion	D_U_L
Users with three posts with 3+ likes each	U_T_T
At least one of contribution	M_O_C
Highest efficiency ratio	M_E_R
Consistent contribution	M_C
Moderator power	M_M
Best technical contribution	T_C


TO DO LIST:

- changing the FORUM CSV files to new ones - checking the final amount 
- modifying the m_m user_ids, and the calculation of the 5% share, based on the number of rewarded participants
- remowing duplications at the of the code 

In [3]:
#proposals and their status
#manually created file
df_proposals = pd.read_csv('dao_v03_1_proposals.csv')
df_proposals

,proposal number,title,topic_id,post_id,post_id_original,like_count,enter_vote_phase,us,user_id,winning,no_vote,vote_stage
0,1,Proposal: HOPR network's SOCKS5 client,3867,11001,10646,10,True,fatcatdog,643,0,0,1
1,2,[Proposal] IoT home security cameras,3868,11003,10236,26,True,emmerson_biggins,603,0,0,1
2,3,Proposal: Implement performance check tool,3869,11007,10736,14,True,fatcatdog,643,0,0,1
3,4,PROPOSAL: Notice and Complaint Dapp,3871,11010,10225,3,False,lesinance,860,0,1,0
4,5,Proposal: Lightweight multipurpose web proxy,3872,11012,10226,13,True,jbradach,1299,0,0,1
5,6,[DAO proposal] Fund/initiate the development o...,3873,11014,10065,40,True,jones_05,1307,0,0,1
6,7,Proposal: HOPR as a basis for smart home services,3874,11017,10869,8,True,ragws13,697,0,0,1
7,8,Proposal - completely anonymous browser on the...,3875,11019,10514,20,True,borissv46,544,1,0,0
8,9,[Proposal] SSL / TLS Replacement,3876,11021,10223,9,False,emmerson_biggins,603,0,1,0
9,10,[Proposal] MetaMask HOPR Snap,3877,11023,10379,38,True,jjperezaguinaga,1525,0,0,1


In [4]:
#creating a column for proposals being considered as valid
df_proposals["ref"] = 1
refer = df_proposals[["post_id", "ref"]]

In [5]:
#signitures given to proposals
df_signitures = pd.read_csv('dao_v03_5_distributions.csv')
df_signitures

,topic_id,post_id,user_id,if_proposer
0,3875,11019,693,NaN
1,3885,11041,45,NaN
2,3878,11025,766,NaN
3,3877,11023,1536,NaN
4,3885,11041,523,NaN
...,...,...,...,...
369,3871,11010,569,NaN
370,3879,11027,38,NaN
371,3867,11001,296,NaN
372,3867,11001,70,NaN


In [6]:
#dropping signatures which are not considered
df_signitures = pd.merge(df_signitures, refer, how="outer", on=["post_id"])

In [7]:
df_signitures = df_signitures[df_signitures['ref'].notna()]
df_signitures.head(2)

,topic_id,post_id,user_id,if_proposer,ref
0,3875,11019,693,NaN,1
1,3875,11019,1296,NaN,1


In [8]:
#mergind the proposal and the signiture files
df_merge_pr_si = pd.merge(df_signitures, df_proposals, how="outer", on=["topic_id"])

In [9]:
# checking the max number of signitures - if someone liked more than 8, must be out
df_merge_pr_si["user_id_x"].value_counts().reset_index()

,index,user_id_x
0,1508,8
1,38,8
2,847,8
3,603,8
4,1166,7
...,...,...
109,1164,1
110,644,1
111,547,1
112,612,1


### WINNING PROPOSAL P-W-P

In [10]:
#winning proposal P_W_P
p_w_p = df_proposals[df_proposals['winning'] == 1].reset_index(drop=True)
p_w_p = p_w_p[['user_id','winning']]
p_w_p

,user_id,winning
0,544,1


In [11]:
#post_id of winning proposal
winner_post_id = [11019]

### WINNING SIGNITORIES P-W-S

In [12]:
#celectingt the signitories which sign the winning proposal
p_w_s = df_signitures.loc[df_signitures['post_id'].isin(winner_post_id)].reset_index(drop=True)

In [13]:
#dropping the proposer from the list of signatories
p_w_s = p_w_s[p_w_s['user_id'] != 544]

In [14]:
p_w_s = p_w_s[['user_id','ref']].rename(columns={'ref': 'winning_signitures'}).reset_index(drop=True)

In [15]:
#summing up by user_id
p_w_s = p_w_s.groupby(by="user_id").sum().reset_index(drop=False)
p_w_s

,user_id,winning_signitures
0,33,1
1,121,1
2,296,1
3,455,1
4,468,1
5,475,1
6,523,1
7,599,1
8,688,1
9,693,1


### VOTE STAGE PROPOSALS - P_V_P

In [136]:
#creating a dataframe from the proposals
df_vote_stage_pr = df_proposals[df_proposals['vote_stage'] == 1].reset_index(drop=True)
df_vote_stage_pr.head(5)

,proposal number,title,topic_id,post_id,post_id_original,like_count,enter_vote_phase,us,user_id,winning,no_vote,vote_stage,ref
0,1,Proposal: HOPR network's SOCKS5 client,3867,11001,10646,10,True,fatcatdog,643,0,0,1,1
1,2,[Proposal] IoT home security cameras,3868,11003,10236,26,True,emmerson_biggins,603,0,0,1,1
2,3,Proposal: Implement performance check tool,3869,11007,10736,14,True,fatcatdog,643,0,0,1,1
3,5,Proposal: Lightweight multipurpose web proxy,3872,11012,10226,13,True,jbradach,1299,0,0,1,1
4,6,[DAO proposal] Fund/initiate the development o...,3873,11014,10065,40,True,jones_05,1307,0,0,1,1


In [17]:
#vote stage proposers
p_v_p = df_vote_stage_pr[['user_id','vote_stage']]

In [18]:
#summing up by user_id
p_v_p = p_v_p.groupby(by="user_id").sum().reset_index(drop=False)
p_v_p

,user_id,vote_stage
0,376,1
1,594,1
2,603,2
3,643,2
4,697,1
5,1103,1
6,1299,1
7,1307,1
8,1508,1
9,1525,1


### VOTE STAGE SIGNITURES - P_V_S

In [19]:
#creating a dataframe from the signitures
df_vote_stage_si = df_signitures.loc[df_signitures['post_id'].isin(df_vote_stage_pr['post_id'])].reset_index(drop=True)

In [135]:
df_vote_stage_si.head(5)

,topic_id,post_id,user_id,if_proposer,ref,combi2
0,3885,11041,45,NaN,1,1104145
1,3885,11041,523,NaN,1,11041523
2,3885,11041,1347,NaN,1,110411347
3,3885,11041,124,NaN,1,11041124
4,3885,11041,672,NaN,1,11041672


In [21]:
# adding a variable, which is a combination of post id and user id, to remove proposers form the signitories
df_vote_stage_si = df_vote_stage_si.assign(combi2 = (df_vote_stage_si['post_id'].apply(lambda x: str(x))) + (df_vote_stage_si['user_id'].apply(lambda x: str(x))))

In [134]:
#list of proposers
p_v_s_proposers = df_vote_stage_pr[['user_id', 'post_id']]
p_v_s_proposers.head(5)

,user_id,post_id
0,643,11001
1,603,11003
2,643,11007
3,1299,11012
4,1307,11014


In [23]:
#adding the same variable to the list of proposers
p_v_s_proposers = p_v_s_proposers.assign(combi2 = (p_v_s_proposers['post_id'].apply(lambda x: str(x))) + (p_v_s_proposers['user_id'].apply(lambda x: str(x))))

In [133]:
p_v_s_proposers.head(5)

,user_id,post_id,combi2
0,643,11001,11001643
1,603,11003,11003603
2,643,11007,11007643
3,1299,11012,110121299
4,1307,11014,110141307


In [25]:
#removing the proposers
vote_signitures_without_proposers = df_vote_stage_si.loc[~df_vote_stage_si['combi2'].isin(p_v_s_proposers['combi2'])].reset_index(drop=True)

In [132]:
vote_signitures_without_proposers.head(5)

,topic_id,post_id,user_id,if_proposer,ref,combi2
0,3885,11041,45,NaN,1,1104145
1,3885,11041,523,NaN,1,11041523
2,3885,11041,1347,NaN,1,110411347
3,3885,11041,124,NaN,1,11041124
4,3885,11041,672,NaN,1,11041672


In [27]:
# signitures given for the not winning vote stage proposals
p_v_s = vote_signitures_without_proposers[['user_id', 'ref']]
p_v_s = p_v_s.rename(columns={'ref': 'vote_stage_signitures'}).reset_index(drop=True)
p_v_s

,user_id,vote_stage_signitures
0,45,1
1,523,1
2,1347,1
3,124,1
4,672,1
...,...,...
272,843,1
273,1299,1
274,70,1
275,510,1


In [28]:
#summing up the signitures
p_v_s = p_v_s.groupby(by="user_id").sum().reset_index(drop=False)
p_v_s

,user_id,vote_stage_signitures
0,9,1
1,33,3
2,35,2
3,38,6
4,45,2
...,...,...
100,1540,5
101,1541,1
102,1543,1
103,1544,1


### REFERENDUM STAGE PROPOSALS - P_R_P

In [29]:
#proposals
df_no_vote_pr = df_proposals[df_proposals['no_vote'] == 1].reset_index(drop=True)

In [30]:
#no_vote stage proposals
p_r_p = df_no_vote_pr[['user_id','no_vote']]

In [31]:
#summing up the proposals by user_id
p_r_p = p_r_p.groupby(by="user_id").sum().reset_index(drop=False)
p_r_p

,user_id,no_vote
0,603,2
1,860,1
2,1504,1


### REFERENDUM STAGE SIGNITURES - P_R_S

In [32]:
df_no_vote_si = df_no_vote_pr[['user_id','post_id','no_vote']]

In [33]:
#creating a variable to remove the proposers
df_no_vote_si = df_no_vote_si.assign(combi2 = (df_no_vote_si['post_id'].apply(lambda x: str(x))) + (df_no_vote_si['user_id'].apply(lambda x: str(x))))

In [34]:
# proposal's post_id
df_no_vote_signitures = df_signitures.loc[df_signitures['post_id'].isin(df_no_vote_pr['post_id'])].reset_index(drop=True)

In [35]:
#creating a variable to remove the proposers
df_no_vote_signitures = df_no_vote_signitures.assign(combi2 = (df_no_vote_signitures['post_id'].apply(lambda x: str(x))) + (df_no_vote_signitures['user_id'].apply(lambda x: str(x))))

In [36]:
#removing proposers
no_vote_signitures_without_proposers = df_no_vote_signitures.loc[~df_no_vote_signitures['combi2'].isin(df_no_vote_si['combi2'])].reset_index(drop=True)

In [137]:
#subfil
p_r_s = no_vote_signitures_without_proposers[['user_id', 'ref']].rename(columns={'ref': 'no_vote_signitures'}).reset_index(drop=True)

In [138]:
#adding up the votes
p_r_s = p_r_s.groupby(by="user_id").sum().reset_index(drop=False)
p_r_s

,user_id,no_vote_signitures
0,35,1
1,38,2
2,124,1
3,480,1
4,544,1
5,569,1
6,588,1
7,697,1
8,729,1
9,754,2


### DISCUSSION (D_P_L, D_U_L, U_T_T)

In [139]:
df_discussion = pd.read_csv('dao_v03_6_discussion.csv', names=['user_id', 'total_allocation', 'd_p_l', 'd_u_l', 'u_t_t'], header=0)

In [140]:
df_discussion.head(5)

,user_id,total_allocation,d_p_l,d_u_l,u_t_t
0,603,0.036149,0.001149,0.01,0.025
1,643,0.036149,0.001149,0.01,0.025
2,1307,0.036149,0.001149,0.01,0.025
3,1525,0.036149,0.001149,0.01,0.025
4,1299,0.011149,0.001149,0.01,NaN


In [141]:
df_discussion['user_id'] = df_discussion['user_id'].fillna(0)

### MISCELLANEOUS_1 - M_O_C 

In [142]:
df_miscellaneous_1 = pd.read_csv('dao_v03_7_miscellaneous_1.csv', names=['user_id', 'm_o_c', 'count'], header=0)
df_miscellaneous_1.head(5)

,user_id,m_o_c,count
0,38,0.001042,96
1,45,0.001042,96
2,48,0.001042,96
3,53,0.001042,96
4,70,0.001042,96


### MISCELLANEOUS_2 M_E_R

In [143]:
df_miscellaneous_2 = pd.read_csv('dao_v03_8_miscellaneous_2.csv', names=['user_id', 'coef', 'm_e_r'], header=0)
df_miscellaneous_2.head(5)

,user_id,coef,m_e_r
0,1155,23000,0.005
1,1322,6000,0.005
2,1504,5000,0.005
3,1525,4272,0.005
4,376,3666,0.005


### MISCELLANEOUS_3 M_C

In [44]:
df_miscellaneous_3 = pd.read_csv('dao_v03_9_miscellaneous_3.csv', names=['user_id', 'post_id', 'm_c', 'count'], header=0)
df_miscellaneous_3

,user_id,post_id,m_c,count
0,603,10205,0.01,7
1,603,10236,0.01,7
2,1155,10090,0.01,7
3,1307,10065,0.01,7
4,1525,10379,0.01,7


In [45]:
df_miscellaneous_3 = df_miscellaneous_3.groupby(by="user_id").sum().reset_index(drop=False)
df_miscellaneous_3

,user_id,post_id,m_c,count
0,603,20441,0.02,14
1,1155,10090,0.01,7
2,1307,10065,0.01,7
3,1525,10379,0.01,7


### MODERATOR POWER M_M

In [144]:
#NEED TO BE MODIFIED, the user_ids were not decided yet!!!
#consider the number of participants in the share (4 participants at the moment)
m_m = pd.DataFrame({'user_id': [603, 594, 1307, 810], 'm_m': [0.05/4, 0.05/4, 0.05/4, 0.05/4]}).reset_index(drop=True)
m_m

,user_id,m_m
0,603,0.0125
1,594,0.0125
2,1307,0.0125
3,810,0.0125


### TECH POWER T_C

In [145]:
t_c = pd.DataFrame({'user_id': [603, 594, 1307, 810], 't_c': [0.1/4, 0.1/4, 0.1/4, 0.1/4]}).reset_index(drop=True)

### WALET AND NFT ADDRESS

In [146]:
wallet_address = pd.read_csv('wallet_address_10.csv', names=['id', 'username_lower', 'user_id', 'add'], header=0)
wallet_address

,id,username_lower,user_id,add
0,1059,niceman,1059.0,0x000006eee6e39015cB523AeBDD4d0B1855aBa682
1,1398,chenk7,1398.0,0x00CcB538dDe93630D442Ce14eBfc367BB0ddcA75
2,1179,ceron,1179.0,0x01f71F2a59D19030bf79961e3e57C82922FeecCa
3,1115,andree,1115.0,0x0305Ff47bA4ecE44Dd9211bF1D1311bb14292bFB
4,1542,guedes,1542.0,0x034e1f7A66b582B68E511B325ED0cCB71bb4bc12
...,...,...,...,...
698,1216,bron,1216.0,Екатеринбург
699,"1273,tatyana6133,1273,""Микрорайон,14""",NaN,NaN,NaN
700,1265,radole,1265.0,м.Рівне в.ч А2798
701,"1316,avkar,1316,""Нахимовский проспект, 23к2-121""",NaN,NaN,NaN


In [147]:
nft_address = pd.read_csv('nft_address_11.csv', names=['id', 'username_lower', 'user_id', 'nft_add'], header=0)
nft_address

,id,username_lower,user_id,nft_add
0,1398,chenk7,1398,0x00CcB538dDe93630D442Ce14eBfc367BB0ddcA75
1,1179,ceron,1179,0x01f71F2a59D19030bf79961e3e57C82922FeecCa
2,1542,guedes,1542,0x034e1f7A66b582B68E511B325ED0cCB71bb4bc12
3,1256,loaderleon,1256,0x03E7fA64781cab468f0501bbd7D6c6EeBe003360
4,1380,nikolaevichdm,1380,0x03F182ec7474877EC8d830a5Fcb804FD132755c1
...,...,...,...,...
367,1265,radole,1265,38
368,1496,kush,1496,borishrbacek@gmail.com
369,1370,manda,1370,china
370,1552,mostafa625000,1552,Fazilat St


In [50]:
#merging them
addresses = pd.merge(wallet_address, nft_address, how="outer", on=["user_id"])

In [148]:
#creating a subfile
addresses = addresses[['user_id', 'add', 'nft_add']]
addresses

,user_id,add,nft_add
0,1059.0,0x000006eee6e39015cB523AeBDD4d0B1855aBa682,NaN
1,1398.0,0x00CcB538dDe93630D442Ce14eBfc367BB0ddcA75,0x00CcB538dDe93630D442Ce14eBfc367BB0ddcA75
2,1179.0,0x01f71F2a59D19030bf79961e3e57C82922FeecCa,0x01f71F2a59D19030bf79961e3e57C82922FeecCa
3,1115.0,0x0305Ff47bA4ecE44Dd9211bF1D1311bb14292bFB,NaN
4,1542.0,0x034e1f7A66b582B68E511B325ED0cCB71bb4bc12,0x034e1f7A66b582B68E511B325ED0cCB71bb4bc12
...,...,...,...
729,1414.0,NaN,0xeEB07faFcb5761394BE481f03F1048aab0deaDC4
730,1475.0,NaN,0xF68500D8Ebc678077A9388Cf0Aa570536613f17C
731,1311.0,NaN,0xfEf5ea9673d8dc1461dC3E6218C3Dbcb3E70B939
732,1273.0,NaN,3


### merging categories and wallets

In [52]:
#creating a list of dataframes from the categories
categories = [p_w_p, p_w_s, p_v_p, p_v_s, p_r_p, p_r_s, df_discussion, df_miscellaneous_1, df_miscellaneous_2, df_miscellaneous_3, m_m, t_c]

In [150]:
#merging each element of the list, after each other, every category can add new user_id
df_merged_0 = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'], how='outer'), categories)
df_merged_0

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,u_t_t,m_o_c,count_x,coef,m_e_r,post_id,m_c,count_y,m_m,t_c
0,544,1.0,NaN,NaN,1.0,NaN,1.0,0.011149,0.001149,0.01,NaN,0.001042,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,121,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,296,NaN,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,455,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1455,NaN,NaN,NaN,NaN,NaN,NaN,0.001149,0.001149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,1364,NaN,NaN,NaN,NaN,NaN,NaN,0.001149,0.001149,NaN,NaN,0.001042,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001042,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001042,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
#merging the dataframe with the addresses - here only the existing user_ids are consideres from the wallet list
df_merged = pd.merge(df_merged_0, addresses, how='left', on=["user_id"])
df_merged.head(5)

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,...,count_x,coef,m_e_r,post_id,m_c,count_y,m_m,t_c,add,nft_add
0,544,1.0,NaN,NaN,1.0,NaN,1.0,0.011149,0.001149,0.01,...,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46
1,33,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8
2,121,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1
3,296,NaN,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x3c58541209301F68C5c9Ac73229954dff07A0784,0x3c58541209301F68C5c9Ac73229954dff07A0784
4,455,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x788A8A180F27f86B2148fbD7836ee6F0794D9296,0x788A8A180F27f86B2148fbD7836ee6F0794D9296


In [56]:
#removing admins - user_ids which are belong to a team member

In [159]:
#also available on the forum
admin = pd.read_csv('admin.csv', names=['user_id', 'usename'], header=0)

In [160]:
#adding a variable with value = 1
admin.loc[:,'admin'] = 1

In [161]:
#merging with the dataframe
df_merged = pd.merge(df_merged, admin, how="left", on=["user_id"])
df_merged

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,...,m_m,t_c,add,nft_add,usename_x,admin_x,usename_y,admin_y,usename,admin
0,544,1.0,NaN,NaN,1.0,NaN,1.0,0.011149,0.001149,0.01,...,NaN,NaN,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46,NaN,NaN,NaN,NaN,NaN,NaN
1,33,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8,NaN,NaN,NaN,NaN,NaN,NaN
2,121,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1,NaN,NaN,NaN,NaN,NaN,NaN
3,296,NaN,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0x3c58541209301F68C5c9Ac73229954dff07A0784,0x3c58541209301F68C5c9Ac73229954dff07A0784,NaN,NaN,NaN,NaN,NaN,NaN
4,455,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0x788A8A180F27f86B2148fbD7836ee6F0794D9296,0x788A8A180F27f86B2148fbD7836ee6F0794D9296,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1455,NaN,NaN,NaN,NaN,NaN,NaN,0.001149,0.001149,NaN,...,NaN,NaN,0xf7c6382Da0A3ee08aD4c201b01E33e122600Ab6f,0xf7c6382Da0A3ee08aD4c201b01E33e122600Ab6f,NaN,NaN,NaN,NaN,NaN,NaN
149,1364,NaN,NaN,NaN,NaN,NaN,NaN,0.001149,0.001149,NaN,...,NaN,NaN,0xe4126Fd057ab2fC08B6cE2621De4C3bc56464101,0xe4126Fd057ab2fC08B6cE2621De4C3bc56464101,NaN,NaN,NaN,NaN,NaN,NaN
150,738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0xF3062faAA61CD4Ee6E8705135A3Bc6419CD766B2,0xF3062faAA61CD4Ee6E8705135A3Bc6419CD766B2,NaN,NaN,NaN,NaN,NaN,NaN
151,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
#NANs changed to 0
df_merged = df_merged.fillna(0)

In [163]:
#if the "admin" category is 1, the given used_id is removed
df_merged = df_merged[df_merged['admin'] != 1]
df_merged

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,...,m_m,t_c,add,nft_add,usename_x,admin_x,usename_y,admin_y,usename,admin
0,544,1.0,0.0,0.0,1.0,0.0,1.0,0.011149,0.001149,0.01,...,0.0,0.0,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46,0x508f1b6C5eb1e0F9d48795C848d87288BaE13a46,0,0.0,0,0.0,0,0.0
1,33,0.0,1.0,0.0,3.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8,0x7d3a7Ad44bFcFc0962709ECE4c149128717906B8,0,0.0,0,0.0,0,0.0
2,121,0.0,1.0,0.0,3.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1,0x28beb6Fd342F7F41c0eBe086F78e43468c7A25C1,0,0.0,0,0.0,0,0.0
3,296,0.0,1.0,0.0,4.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0x3c58541209301F68C5c9Ac73229954dff07A0784,0x3c58541209301F68C5c9Ac73229954dff07A0784,0,0.0,0,0.0,0,0.0
4,455,0.0,1.0,0.0,5.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0x788A8A180F27f86B2148fbD7836ee6F0794D9296,0x788A8A180F27f86B2148fbD7836ee6F0794D9296,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1455,0.0,0.0,0.0,0.0,0.0,0.0,0.001149,0.001149,0.00,...,0.0,0.0,0xf7c6382Da0A3ee08aD4c201b01E33e122600Ab6f,0xf7c6382Da0A3ee08aD4c201b01E33e122600Ab6f,0,0.0,0,0.0,0,0.0
149,1364,0.0,0.0,0.0,0.0,0.0,0.0,0.001149,0.001149,0.00,...,0.0,0.0,0xe4126Fd057ab2fC08B6cE2621De4C3bc56464101,0xe4126Fd057ab2fC08B6cE2621De4C3bc56464101,0,0.0,0,0.0,0,0.0
150,738,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0xF3062faAA61CD4Ee6E8705135A3Bc6419CD766B2,0xF3062faAA61CD4Ee6E8705135A3Bc6419CD766B2,0,0.0,0,0.0,0,0.0
151,949,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.0,0.0,0,0,0,0.0,0,0.0,0,0.0


### calculating coef in the proposal section - based on the allocation bard

In [177]:
df_merged['p_w_p'] = df_merged['winning']*0.05

In [165]:
#the allocations are shared between the users in the category, e.g. p_w_s if someone signed the winning proposal, gets the proportional share of the 5%

In [166]:
df_merged['p_w_s'] = 0.05/sum(p_w_s["winning_signitures"])*df_merged['winning_signitures']

In [167]:
df_merged['p_v_p'] = 0.09/sum(p_v_p["vote_stage"])*df_merged['vote_stage']

In [168]:
df_merged['p_v_s'] = 0.06/sum(p_v_s['vote_stage_signitures'])*df_merged['vote_stage_signitures']

In [169]:
df_merged['p_r_p'] = 0.03/sum(p_r_p['no_vote'])*df_merged['no_vote']

In [170]:
df_merged['p_r_s'] = 0.02/sum(p_r_s["no_vote_signitures"])*df_merged['no_vote_signitures']

### calculating total share

In [172]:
#sum of the proportional shares
df_merged['total_coef'] = df_merged['p_w_p'] + df_merged['p_w_s'] + df_merged['p_v_p'] + df_merged['p_v_s'] + df_merged['p_r_p'] + df_merged['p_r_s'] + df_merged['d_p_l'] + df_merged['d_u_l'] + df_merged['u_t_t'] + df_merged['m_o_c'] + df_merged["m_e_r"] + df_merged["m_c"] + df_merged['m_m'] + df_merged['t_c']

In [173]:
#the proportional share from the total reward
df_merged['score'] = df_merged["total_coef"]*25000

In [175]:
#sorting by the ammount of the reward
df_merged = (df_merged.sort_values(by='score', ascending=False)).reset_index(drop=True)
df_merged.head()

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,...,usename,admin,p_w_p,p_w_s,p_v_p,p_v_s,p_r_p,p_r_s,total_coef,score
0,603,0.0,0.0,2.0,0.0,2.0,0.0,0.036149,0.001149,0.01,...,0,0.0,0.00,0.0,0.0150,0.000000,0.015,0.000000,0.129691,3242.277299
1,1307,0.0,0.0,1.0,3.0,0.0,0.0,0.036149,0.001149,0.01,...,0,0.0,0.00,0.0,0.0075,0.000650,0.000,0.000000,0.097841,2446.022786
2,544,1.0,0.0,0.0,1.0,0.0,1.0,0.011149,0.001149,0.01,...,0,0.0,0.05,0.0,0.0000,0.000217,0.000,0.000833,0.063241,1581.025795
3,1525,0.0,0.0,1.0,0.0,0.0,0.0,0.036149,0.001149,0.01,...,0,0.0,0.00,0.0,0.0075,0.000000,0.000,0.000000,0.059691,1492.277299
4,643,0.0,0.0,2.0,0.0,0.0,0.0,0.036149,0.001149,0.01,...,0,0.0,0.00,0.0,0.0150,0.000000,0.000,0.000000,0.057191,1429.777299


In [176]:
#checkpoint - it should be 25000 minimal difference can be caused by the python
sum(df_merged['score'])

23744.58483754513

In [178]:
#users without score
empty_wal = df_merged[df_merged['score'] == 0 ].index

In [76]:
#dropping the users with no distribution
df_merged.drop(empty_wal, inplace=True)

### binning and tagging with NFT status, based on the score

In [179]:
#creating label
b_l = ["Bronze", "Silver", "Gold", "Diamond"]
#using binning function to creat groups
df_merged["NFT_status"] = pd.qcut(df_merged["score"], 4, labels=b_l)

In [180]:
df_merged["NFT_status"].value_counts()

Silver     48
Bronze     38
Diamond    33
Gold       33
Name: NFT_status, dtype: int64

### voting

In [186]:
#url and query 
url = 'https://hub.snapshot.org/graphql?operationName=Votes'

data_h = '''
query Votes {
  votes (
    first: 1000
    skip: 0
    where: {
      proposal: "0xe02cd1e68980a1572f03de7d849c5b6c86620d6774f3a652f11069d2ad290ae9"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    voter
    created
    proposal {
      id
    }
    choice
    space {
      id
    }
  }
}
'''

response_h = requests.post(url,json={'query': data_h})
#dictionary from the query
data_h = json.loads(response_h.text)

In [187]:
#creating a datafram from the dictionary
df_h = pd.DataFrame((pd.DataFrame(data_h)).iloc[0][0])
df_h.head()

,id,voter,created,proposal,choice,space
0,0x416daa4777942079a6ad257a231e8ebb06f03afa46cd...,0x077C5d343e226E7e152246780be2BCa2AF91631e,1647697466,{'id': '0xe02cd1e68980a1572f03de7d849c5b6c8662...,"{'5': 3000, '9': 1000}",{'id': 'v03.hopr-dao.eth'}
1,0xf8256beb3b7a4c9f1eb825ace4204a9ec0aa62193aac...,0x72eCB0E30C261aA219a7A99db04526cA9E71D5b9,1647695988,{'id': '0xe02cd1e68980a1572f03de7d849c5b6c8662...,"{'1': 1, '6': 1}",{'id': 'v03.hopr-dao.eth'}
2,0x37880f7b434458e6c664606a2a3faa62b65ab4123fab...,0xEccC484DD4093F62fC24c4725800013FBc2C1D15,1647694139,{'id': '0xe02cd1e68980a1572f03de7d849c5b6c8662...,{'1': 1},{'id': 'v03.hopr-dao.eth'}
3,0x2677679498fe7ec65a7f7ab12f4cea1624beec42a6f6...,0x20D236B1800f2B597Cd64D920b71D36b87991f30,1647692593,{'id': '0xe02cd1e68980a1572f03de7d849c5b6c8662...,{'6': 1},{'id': 'v03.hopr-dao.eth'}
4,0xe9b71a3327b834c5e32b83a5af1d80666c64bf88d382...,0x62f2F6B32454A14809170F1B61914a6c0DE80CF8,1647690354,{'id': '0xe02cd1e68980a1572f03de7d849c5b6c8662...,{'2': 1},{'id': 'v03.hopr-dao.eth'}


In [188]:
from pandas.io.json import json_normalize

In [190]:
#separating the answers from the vote - which are still dictionary objects
sample = df_h['choice']

In [192]:
#normalizing the answers
choices = pd.json_normalize(sample)
choices.head(5)

,5,9,1,6,2,3,4,8,12,7,10,11,13
0,3000.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
#merging back the answers to the dataframe
df_snapshot = pd.concat([df_h, choices], axis=1)

In [196]:
#separating the voter's address
vote = pd.DataFrame(df_snapshot['voter'])

In [197]:
#changing the characters to lower case
vote['voter'] = vote['voter'].str.lower()

In [198]:
#changing the name of the column
vote.columns = ["nft_add"]

In [91]:
#adding a variable as identifier after merging with the forum data
vote["voter"] = 1 
vote

,nft_add,voter
0,0x077c5d343e226e7e152246780be2bca2af91631e,1
1,0x72ecb0e30c261aa219a7a99db04526ca9e71d5b9,1
2,0xeccc484dd4093f62fc24c4725800013fbc2c1d15,1
3,0x20d236b1800f2b597cd64d920b71d36b87991f30,1
4,0x62f2f6b32454a14809170f1b61914a6c0de80cf8,1
...,...,...
260,0x6fb3071447c444e51f7b282a76f06f811e21d3ec,1
261,0x0d36dd40ff721ac6e3f40fc3e36a5566609a03f1,1
262,0xbf45dd1bca8067769494b935c69763ca29de85a0,1
263,0x83f097475c0d5feaaea43cd42781d4313611a0d9,1


In [199]:
#changing the forum addresses to lower case
df_merged['nft_add'] = df_merged['nft_add'].str.lower()

In [200]:
df_merged['add'] = df_merged['add'].str.lower()

### merging the addresses from the forum data and from the snapshot

In [201]:
df_merged_vote = pd.merge(df_merged, vote, how="outer", on=["nft_add"])
df_merged_vote.head(5)

,user_id,winning,winning_signitures,vote_stage,vote_stage_signitures,no_vote,no_vote_signitures,total_allocation,d_p_l,d_u_l,...,admin,p_w_p,p_w_s,p_v_p,p_v_s,p_r_p,p_r_s,total_coef,score,NFT_status
0,603.0,0.0,0.0,2.0,0.0,2.0,0.0,0.036149,0.001149,0.01,...,0.0,0.00,0.0,0.0150,0.000000,0.015,0.000000,0.129691,3242.277299,Diamond
1,1307.0,0.0,0.0,1.0,3.0,0.0,0.0,0.036149,0.001149,0.01,...,0.0,0.00,0.0,0.0075,0.000650,0.000,0.000000,0.097841,2446.022786,Diamond
2,544.0,1.0,0.0,0.0,1.0,0.0,1.0,0.011149,0.001149,0.01,...,0.0,0.05,0.0,0.0000,0.000217,0.000,0.000833,0.063241,1581.025795,Diamond
3,1525.0,0.0,0.0,1.0,0.0,0.0,0.0,0.036149,0.001149,0.01,...,0.0,0.00,0.0,0.0075,0.000000,0.000,0.000000,0.059691,1492.277299,Diamond
4,643.0,0.0,0.0,2.0,0.0,0.0,0.0,0.036149,0.001149,0.01,...,0.0,0.00,0.0,0.0150,0.000000,0.000,0.000000,0.057191,1429.777299,Diamond


In [202]:
#counting the addresses, which are showing up only in the voting
df_merged_vote['NFT_status'].isna().sum()

180

### giving every user at least "bronze" status - because the voting participants did not necessarily participated in the forum discussion

In [203]:
df_merged_vote['NFT_status'] = df_merged_vote["NFT_status"].fillna('Bronze')

In [204]:
df_merged_vote['NFT_status'].value_counts()

Bronze     218
Silver      48
Diamond     33
Gold        33
Name: NFT_status, dtype: int64

In [205]:
#changing NAN to 0
df_merged_vote['score'].fillna(0)

0      3242.277299
1      2446.022786
2      1581.025795
3      1492.277299
4      1429.777299
          ...     
327       0.000000
328       0.000000
329       0.000000
330       0.000000
331       0.000000
Name: score, Length: 332, dtype: float64

### filling up the missing addresses (mutually) - if the participant didn't have wallet address, but had NTF address, it's inserted and vica versa

In [210]:
# 0 changed to NAN
df_merged_vote['add'] = df_merged_vote['add'].replace({0:np.nan})

In [211]:
# NAN replaced with the other address
df_merged_vote['add'] = df_merged_vote['add'].fillna(df_merged_vote['nft_add'])

In [212]:
# 0 changed to NAN
df_merged_vote['nft_add'] = df_merged_vote['nft_add'].replace({0:np.nan})

In [213]:
# NAN replaced with the other address
df_merged_vote['nft_add'] = df_merged_vote['nft_add'].fillna(df_merged_vote['add'])

In [224]:
#sorting values
df_merged_vote = (df_merged_vote.sort_values(by='score', ascending=False)).reset_index(drop=True)

### check duplicates and drop them, if it's necessary

df_merged_vote['add'].value_counts()

df_merged_vote.drop_duplicates(subset=['add'])

### creating the list for HOPR distribution

In [215]:
#creating HOPR token format from the score
df_merged_vote['score_h'] = df_merged_vote['score']*1e18

In [218]:
#subfile for HOPR allocation
hopr_allocation = df_merged_vote.filter(['add', "score", "score_h",], axis=1)

In [225]:
#dropping NANs
hopr_allocation = hopr_allocation.dropna()

In [226]:
#changint the datatype of the score to int
hopr_allocation["score_h"] = hopr_allocation['score_h'].apply(lambda x: int(x))
hopr_allocation

,add,score,score_h
0,0xeb4872b4c995f6613077090438eae69cfeff142f,3242.277299,3242277298850574630912
1,0x09e3c4bf50a15225d4087fe089282cb5506dcce6,2446.022786,2446022786215195443200
2,0x508f1b6c5eb1e0f9d48795c848d87288bae13a46,1581.025795,1581025794638781612032
3,0x6ab26a36e97a4d25e7256162c4e5a633b0e65e34,1492.277299,1492277298850574893056
4,0x2a8b74f9d2bfdc204f3b29fec435e8a4f8a7f6fa,1429.777299,1429777298850574761984
...,...,...,...
147,0xba1cad8e212e79cd1b4298abe09eb6b44384ffa2,5.415162,5415162454873646080
148,0x938f10800b4d2b558026dce858fe58b615288361,5.415162,5415162454873646080
149,0xfc91fc0ccbd62c9d0517e5a6b337c99109292b13,5.415162,5415162454873646080
150,0x05c254989526b621ced514f5dad9123fe4b803fd,5.415162,5415162454873646080


In [228]:
#creating a new variable from the score as str 
hopr_allocation["score_sth"] = hopr_allocation['score_h'].apply(lambda x: str(x))

In [231]:
#creating an array from the ammount
hopr_dist_score_h = hd['score_sth'].to_numpy()

In [232]:
#creating an array from the address
hopr_dist_add = hd['add'].to_numpy()

### NFT distribution

In [239]:
#subframe
nft_distribution = df_merged_vote.filter(['nft_add', "NFT_status"], axis=1)

In [240]:
#creating a column with the right format
nft_distribution['eoa'] = '>' + nft_distribution["nft_add"] + '<'

In [241]:
#adding a column as for the status
nft_distribution['grade'] = nft_distribution['NFT_status']

In [242]:
#keeping only the necessary columns
nft_distribution = nft_distribution.drop(['nft_add', 'NFT_status'], axis=1)

In [243]:
nft_distribution = nft_distribution.dropna()
nft_distribution

,eoa,grade
0,>0xeb4872b4c995f6613077090438eae69cfeff142f<,Diamond
1,>0x09e3c4bf50a15225d4087fe089282cb5506dcce6<,Diamond
2,>0x508f1b6c5eb1e0f9d48795c848d87288bae13a46<,Diamond
3,>0x6ab26a36e97a4d25e7256162c4e5a633b0e65e34<,Diamond
4,>0x2a8b74f9d2bfdc204f3b29fec435e8a4f8a7f6fa<,Diamond
...,...,...
327,>0x88a41b1fbba8ec0d983987e2f3c53b4e61efc65f<,Bronze
328,>0xb634eb031955130d3191cb734f9f774d3dfb93ea<,Bronze
329,>0x4a2dd4b190b6dee6939709278b659ed5ffcb9d92<,Bronze
330,>0xbf45dd1bca8067769494b935c69763ca29de85a0<,Bronze


In [244]:
#saving
nft_distribution.to_csv('daov2_nft_distribution.csv')